In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ray on Vertex AI cluster management


<table align="left">

  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ray_on_vertex_ai/ray_cluster_management.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> <br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fray_on_vertex_ai%2Fray_cluster_management.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"> <br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/ray_on_vertex_ai/ray_cluster_management.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> <br>
      Open in Vertex AI Workbench
    </a>
  </td>
<td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ray_on_vertex_ai/ray_cluster_management.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> <br>
      View on GitHub
    </a>
  </td>
</table>

## Overview

This tutorial demonstrates how to use Ray on Vertex AI SDK for cluster management.

Learn more about [Ray on Vertex AI overview](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/overview).

### Objective

In this tutorial, you learn how to create a cluster, list existing clusters, get a cluster, update (manually scaling) a cluster, and delete a cluster.

This tutorial uses the following Vertex AI services and resources:

- [Ray on Vertex AI](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/overview)


The steps performed include:

- Create a cluster.
- List existing clusters.
- Get a cluster.
- Manually scale up the cluster, then scale down the cluster.
- Delete existing clusters.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Ray on Vertex AI pricing](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/overview#pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Getting Started

### Install Vertex AI SDK and other required packages

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform[ray]

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Retrieve the project number
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

### Set network information

[Set up a VPC peering network](https://cloud.google.com/vertex-ai/docs/general/vpc-peering) and private services connection to access Vertex AI.

In [ ]:
VPC_NETWORK = "[your-network-name]"  # @param {type:"string"}
VPC_NETWORK_FULL = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, VPC_NETWORK)
VPC_NETWORK_FULL

### Import libraries

In [ ]:
import vertex_ray

## Create a cluster

Note that within the same VPC network, IP ranges restrict the numbers of clusters and nodes you can create.

In [ ]:
head_node_type = vertex_ray.Resources(
    machine_type="n1-standard-16",
    node_count=1,
)

worker_node_types = [
    vertex_ray.Resources(
        machine_type="n1-standard-8",
        node_count=2,  # Can be > 1
        accelerator_type="NVIDIA_TESLA_T4",
        accelerator_count=1,
    )
]

cluster_resource_name = vertex_ray.create_ray_cluster(
    head_node_type=head_node_type,
    network=VPC_NETWORK_FULL,
    worker_node_types=worker_node_types,
)

## List existing clusters

In [ ]:
clusters = vertex_ray.list_ray_clusters()
clusters

## Update an existing cluster (manually scaling)


Note that the maximum number of worker nodes you can scale up depends on the initial node counts (more details are in these [formulas](https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/set-up)) and is restricted by IP ranges within the same VPC network.

Get the cluster you want to scale.

In [ ]:
cluster = vertex_ray.get_ray_cluster(cluster_resource_name)
cluster

Scale down workers from 2 nodes to 1 node.

In [ ]:
new_worker_node_types = []
for worker_node_type in cluster.worker_node_types:
    worker_node_type.node_count = 1
    new_worker_node_types.append(worker_node_type)

cluster_resource_name = vertex_ray.update_ray_cluster(
    cluster_resource_name=cluster_resource_name,
    worker_node_types=new_worker_node_types,
)

Verify if the cluster is successfully scaled down.

In [ ]:
cluster = vertex_ray.get_ray_cluster(cluster_resource_name)
cluster

Scale up to 2 worker nodes.

In [ ]:
new_worker_node_types = []
for worker_node_type in cluster.worker_node_types:
    worker_node_type.node_count = 2
    new_worker_node_types.append(worker_node_type)

cluster_resource_name = vertex_ray.update_ray_cluster(
    cluster_resource_name=cluster_resource_name,
    worker_node_types=new_worker_node_types,
)

Verify that the cluster is successfully scaled up.

In [ ]:
cluster = vertex_ray.get_ray_cluster(cluster_resource_name)
cluster

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the cluster you created in this tutorial.

In [ ]:
# Delete the cluster
vertex_ray.delete_ray_cluster(cluster.cluster_resource_name)